In [1]:
import torch
import h5py
import os
import numpy as np
from torch import nn as nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
from d2l import torch as d2l
import matplotlib.pyplot as plt
from collections import OrderedDict

In [2]:
# CUDA support 
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
# get the file name of path
def get_fname(path):
    files = os.listdir(path)
    files.sort()
    return files

In [4]:
def get_fpath(path, fname):
    return os.path.join(path, fname)

def get_hdf5_group(hdf5File):
    group = []
    for key in hdf5File:
        group.append(key)
    return group

In [5]:
# ids, masses, density, smoothingLength, coordinates, Velocities(x)

In [6]:
# the deep neural network
class DNN(nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        
        # parameters
        self.depth = len(layers) - 1
        
        # set up layer order dict
        self.activation = nn.Tanh
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(
                ('layer_%d' % i, nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), nn.Linear(layers[-2], layers[-1]))
        )
        layerDict = OrderedDict(layer_list)
        
        # deploy layers
        self.layers = nn.Sequential(layerDict)
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [7]:
# the physics-guided neural network
class PhysicsInformedNN():
    def __init__(self, X, u, layers, lb, ub):
        
        # boundary conditions
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)
        
        # data
        self.x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        self.t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)
        self.u = torch.tensor(u).float().to(device)
        
        # settings
        self.lambda_1 = torch.tensor([0.0], requires_grad=True).to(device)
        self.lambda_2 = torch.tensor([-6.0], requires_grad=True).to(device)
        
        self.lambda_1 = torch.nn.Parameter(self.lambda_1)
        self.lambda_2 = torch.nn.Parameter(self.lambda_2)
        
        # deep neural networks
        self.dnn = DNN(layers).to(device)
        self.dnn.register_parameter('lambda_1', self.lambda_1)
        self.dnn.register_parameter('lambda_2', self.lambda_2)
        
         # optimizers: using the same settings
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"       # can be "strong_wolfe"
        )
        
        self.optimizer_Adam = torch.optim.Adam(self.dnn.parameters())
        self.iter = 0
        
    def net_u(self, x, t):  
        u = self.dnn(torch.cat([x, t], dim=1))
        return u
    
    def net_f(self, x, t):
        """ The pytorch autograd version of calculating residual """
        lambda_1 = self.lambda_1        
        lambda_2 = torch.exp(self.lambda_2)
        u = self.net_u(x, t)
        
        u_t = torch.autograd.grad(
            u, t, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_x = torch.autograd.grad(
            u, x, 
            grad_outputs=torch.ones_like(u),
            retain_graph=True,
            create_graph=True
        )[0]
        u_xx = torch.autograd.grad(
            u_x, x, 
            grad_outputs=torch.ones_like(u_x),
            retain_graph=True,
            create_graph=True
        )[0]
        
        f = u_t + lambda_1 * u * u_x - lambda_2 * u_xx
        return f
    
    def loss_func(self):
        u_pred = self.net_u(self.x, self.t)
        f_pred = self.net_f(self.x, self.t)
        loss = torch.mean((self.u - u_pred) ** 2) + torch.mean(f_pred ** 2)
        self.optimizer.zero_grad()
        loss.backward()
        
        self.iter += 1
        if self.iter % 100 == 0:
            print(
                'Loss: %e, l1: %.5f, l2: %.5f' % 
                (
                    loss.item(), 
                    self.lambda_1.item(), 
                    torch.exp(self.lambda_2.detach()).item()
                )
            )
        return loss
    
    def train(self, nIter):
        self.dnn.train()
        for epoch in range(nIter):
            u_pred = self.net_u(self.x, self.t)
            f_pred = self.net_f(self.x, self.t)
            loss = torch.mean((self.u - u_pred) ** 2) + torch.mean(f_pred ** 2)
            
            # Backward and optimize
            self.optimizer_Adam.zero_grad()
            loss.backward()
            self.optimizer_Adam.step()
            
            if epoch % 10000 == 0:
                print(
                    'It: %d, Loss: %.3e, Lambda_1: %.3f, Lambda_2: %.6f' % 
                    (
                        epoch, 
                        loss.item(), 
                        self.lambda_1.item(), 
                        torch.exp(self.lambda_2).item()
                    )
                )
                
        # Backward and optimize
        self.optimizer.step(self.loss_func)
    
    def predict(self, X):
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)

        self.dnn.eval()
        u = self.net_u(x, t)
        f = self.net_f(x, t)
        u = u.detach().cpu().numpy()
        f = f.detach().cpu().numpy()
        return u, f

In [8]:
nu = 0.01/np.pi

N_u = 500
layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]

dataset = h5py.File("../../datasets_PINNs/dataset_afterDeal/train/particleID1000.hdf5")
x = dataset["dataset"][:, 4].flatten()[:,None]
t = np.linspace(0, len(x), len(x)).flatten()[:,None]
Exact = dataset["dataset"][:, 5].T

X_star = np.concatenate([x,t], axis=1)
u_star = Exact.flatten()[:,None]

# Doman bounds
lb = X_star.min(0)
ub = X_star.max(0) 

In [ ]:
%%time

noise = 0.0            

# create training set
idx = np.random.choice(X_star.shape[0], N_u, replace=False)
X_u_train = X_star[idx,:]
u_train = u_star[idx,:]

# training
model = PhysicsInformedNN(X_u_train, u_train, layers, lb, ub)
model.train(500000)

It: 0, Loss: 5.827e+08, Lambda_1: 0.000, Lambda_2: 0.002479
It: 10000, Loss: 5.815e+08, Lambda_1: 0.000, Lambda_2: 0.002479
It: 20000, Loss: 5.805e+08, Lambda_1: 0.000, Lambda_2: 0.002479
It: 30000, Loss: 5.795e+08, Lambda_1: 0.000, Lambda_2: 0.002479
It: 40000, Loss: 5.786e+08, Lambda_1: 0.000, Lambda_2: 0.002479
It: 50000, Loss: 5.778e+08, Lambda_1: 0.000, Lambda_2: 0.002479
It: 60000, Loss: 5.771e+08, Lambda_1: 0.000, Lambda_2: 0.002479
It: 70000, Loss: 5.765e+08, Lambda_1: 0.000, Lambda_2: 0.002479
It: 80000, Loss: 5.759e+08, Lambda_1: 0.000, Lambda_2: 0.002479
It: 90000, Loss: 5.755e+08, Lambda_1: 0.000, Lambda_2: 0.002479
It: 100000, Loss: 5.751e+08, Lambda_1: 0.000, Lambda_2: 0.002479
It: 110000, Loss: 5.749e+08, Lambda_1: 0.000, Lambda_2: 0.002479
It: 120000, Loss: 5.747e+08, Lambda_1: 0.000, Lambda_2: 0.002479
It: 130000, Loss: 5.746e+08, Lambda_1: 0.000, Lambda_2: 0.002479
It: 140000, Loss: 5.745e+08, Lambda_1: 0.000, Lambda_2: 0.002479
It: 150000, Loss: 5.745e+08, Lambda_1: 